1. Buatlah sebuah Sistem REST API dengan menggunakan dua data di atas
2. Buatlah 1 tabel tambahan yang mendukung backend aplikasi kalian berdasar pada dua berkas di atas. (Silahkan masukkan 3 tabel tersebut dalam MySQL lokal dan berkreasi dengannya)
3. Beberapa berkas yang harus ada
a. models
b. router
4. Untuk berkas lain jika dirasa dibutuhkan silahkan tambahkan, misal schema, berkas utama (baik diberi nama main atau app), dll (konfigurasi .env sesuaikan dengan materi)
5. Setiap Tabel harus ada relasi dengan minimal satu tabel lainnya.
6. Gunakan ORM
7. Silahkan memilih menggunakan Flask atau FastAPI
8. Ada beberapa endpoint api yang harus ada
9. Bisa melihat semua barang
10. Bisa melihat barang dengan kata kunci (jika dicari buku, maka semua barang yang ada kata "buku" akan masuk pencarian)
11. Silahkan tambahkan endpoint lain sesuai kebutuhan
12. Aplikasikan penggunaan JWT, contohnya user hanya bisa melihat data transaksi user tersebut tapi tidak user lain (hanya contoh). product tidak perlu JWT (karena semua user bisa melihat semua product)
13. Dokumetasikan API kalian dengan Swagger
14. Semua tambahan dari ketentuan di atas akan menjadi nilai tambah
15. Jelaskan tentang aplikasi backend kalian pada readme.md

In [1]:
import pandas as pd

df_users = pd.read_csv('users.csv')

display(df_users)

,customer_id,name,city,state,postal
0,1,Claire Gute,Fremont,Nebraska,68025
1,2,Sean O'Donnell,Provo,Utah,84604
2,3,Brosina Hoffman,Keller,Texas,76248
3,4,Andrew Allen,Harlingen,Texas,78550
4,5,Irene Maddox,Kenner,Louisiana,70065
...,...,...,...,...,...
404,405,Ionia McGrath,Plainfield,New Jersey,7060
405,406,Anemone Ratner,Mission Viejo,California,92691
406,407,Sung Chung,Abilene,Texas,79605
407,408,Ricardo Emerson,Jackson,Michigan,49201


In [2]:
df_products = pd.read_csv('products.csv', sep=';', encoding='latin-1')
df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']
display(df_products)

,product_id,product_name,category,sub_category
0,1,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,2,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,3,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,4,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,5,Eldon Fold 'N Roll Cart System,Office Supplies,Storage
...,...,...,...,...
1845,1846,RCA ViSYS 25425RE1 Corded phone,Technology,Phones
1846,1847,Cisco 8961 IP Phone Charcoal,Technology,Machines
1847,1848,"Eureka Hand Vacuum, Bagless",Office Supplies,Appliances
1848,1849,LG G2,Technology,Phones


In [3]:
import random
from datetime import datetime, timedelta

random.seed(7)

def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YYYY format
    formatted_date = random_date.strftime("%d-%m-%Y")
    return formatted_date

n = 1000 #banyaknya kolom

df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
display(df_purchase)

,date,name,product_name,quantity
0,19-08-2020,Aimee Bixby,GBC DocuBind TL300 Electric Binding System,8
1,10-09-2018,Philip Fox,Super Decoflex Portable Personal File,1
2,06-06-2021,Patrick O'Donnell,Ibico Hi-Tech Manual Binding System,9
3,17-07-2017,Frank Olsen,Newell 341,1
4,24-10-2017,Jennifer Jackson,Bevis Round Conference Room Tables and Bases,9
...,...,...,...,...
995,16-02-2017,Magdelene Morse,"Recycled Desk Saver Line ""While You Were Out"" ...",21
996,02-09-2020,Bobby Trafton,Logitech G19 Programmable Gaming Keyboard,16
997,03-08-2021,Julia Dunbar,Xerox 1917,16
998,03-03-2021,Logan Haushalter,Xerox 1910,12


In [4]:
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, and_
from sqlalchemy.ext.declarative import declarative_base
from dotenv import dotenv_values
import urllib.parse

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)
    
    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)
    
    Base.metadata.create_all(engine)
except Exception as e:
    print(e)    

2022-12-23 18:45:24,097 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 18:45:24,099 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 18:45:24,102 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 18:45:24,103 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 18:45:24,105 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 18:45:24,106 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 18:45:24,108 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 18:45:24,109 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 18:45:24,110 INFO sqlalchemy.engine.Engine [generated in 0.00139s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 18:45:24,113 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 18:45:2

In [ ]:
sudah_menambahkan_data_pada_sql = True # Jawab True atau False

if sudah_menambahkan_data_pada_sql:
    pass
else:
    # Insert data ke tabel products
    df_products.to_sql('products', engine, if_exists='replace', index=False)

    # Insert data ke tabel users
    df_users.to_sql('users', engine, if_exists='replace', index=False)

    # Insert data ke tabel purchase
    df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)

2022-12-23 18:45:39,339 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 18:45:39,340 INFO sqlalchemy.engine.Engine [cached since 15.23s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 18:45:39,346 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 18:45:39,346 INFO sqlalchemy.engine.Engine [cached since 15.24s ago] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 18:45:39,350 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `penjualan_uzu`
2022-12-23 18:45:39,351 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 18:45:39,363 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `products`
2022-12-23 18:45:39,364 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 18:45:39,367 INFO sqlalchemy.engine.Engine BEGIN (implicit

In [6]:
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

In [7]:
result = session.query(Products.product_name).all()

nama_barang = 'usb'
sort_desc = 'false'

produk = [i[0] for i in result]
cari_barang = []
for i in produk:
    if nama_barang.lower() in i.lower():
         cari_barang.append(i)
            
if sort_desc==None:
    pass
elif sort_desc.lower() == 'true':
    cari_barang.sort(reverse=False)
elif sort_desc.lower() == 'false':
    cari_barang.sort(reverse=True)
cari_barang

2022-12-23 18:45:24,213 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 18:45:24,218 INFO sqlalchemy.engine.Engine SELECT products.product_name AS products_product_name 
FROM products
2022-12-23 18:45:24,219 INFO sqlalchemy.engine.Engine [generated in 0.00168s] {}


[]

In [8]:
result = (session.query(sa.distinct(Users.name), 
                    Users.city, 
                    Users.state)
.order_by(Users.name.desc()))

hasil = []
for x, i in enumerate(result):
    hasil.append(f'{x+1}. {i[0]} {i[1]} {i[2]}')
hasil

2022-12-23 18:45:24,235 INFO sqlalchemy.engine.Engine SELECT DISTINCT users.name, users.city AS users_city, users.state AS users_state 
FROM users ORDER BY users.name DESC
2022-12-23 18:45:24,236 INFO sqlalchemy.engine.Engine [generated in 0.00159s] {}


['1. Zuschuss Donatelli Taylor Michigan',
 '2. Zuschuss Carroll East Orange New Jersey',
 '3. Xylona Preis Detroit Michigan',
 '4. William Brown Pensacola Florida',
 '5. Vivian Mathis Costa Mesa California',
 '6. Vivek Sundaresam Jonesboro Arkansas',
 '7. Vivek Gonzalez Lowell Massachusetts',
 '8. Valerie Dominguez Rome New York',
 '9. Trudy Schmidt Watertown New York',
 '10. Trudy Glocke Broomfield Colorado',
 '11. Trudy Brown Murray Kentucky',
 '12. Troy Staebel Lake Charles Louisiana',
 '13. Troy Blackwell North Charleston South Carolina',
 '14. Tracy Zic Harlingen Texas',
 '15. Tracy Blumstein Lorain Ohio',
 '16. Tony Sayre Saint Charles Missouri',
 '17. Tony Molinari Highland Park Illinois',
 '18. Tom Prescott Utica New York',
 '19. Tom Boeckenhauer Pearland Texas',
 '20. Toby Swindell Arlington Virginia',
 '21. Toby Ritter Bullhead City Arizona',
 '22. Toby Gnade Austin Texas',
 '23. Toby Carlisle Lake Forest California',
 '24. Toby Braunhardt Jackson Michigan',
 '25. Tim Brockma

In [9]:
n = 25

result = (
session.query(Purchase.name, 
              sa.func.group_concat(Purchase.product_name, separator=", ").label("product_name"), 
              sa.func.sum(Purchase.quantity).label("quantity"))
    .group_by(Purchase.name)
    .order_by(sa.func.sum(Purchase.quantity).desc())
    .limit(n)
    )

hasil = []
for i in result:
    kucing = {}
    kucing['name'] = i[0]
    kucing['product_name'] = i[1].split(',')
    kucing['quantity'] = i[2]
    hasil.append(kucing)
    
hasil

2022-12-23 18:45:24,305 INFO sqlalchemy.engine.Engine SELECT purchase.name AS purchase_name, group_concat(purchase.product_name) AS product_name, sum(purchase.quantity) AS quantity 
FROM purchase GROUP BY purchase.name ORDER BY sum(purchase.quantity) DESC 
 LIMIT %(param_1)s
2022-12-23 18:45:24,306 INFO sqlalchemy.engine.Engine [generated in 0.00221s] {'param_1': 25}


[{'name': 'Debra Catini',
  'product_name': ['Kensington Expert Mouse Optical USB Trackball for PC or Mac',
   'Avery 492',
   'GBC Poly Designer Binding Covers',
   'netTALK DUO VoIP Telephone Service',
   'Aastra 6757i CT Wireless VoIP phone',
   'Boston 16750 Black Compact Battery Pencil Sharpener'],
  'quantity': Decimal('107')},
 {'name': 'Craig Carreira',
  'product_name': ['Jackery Bar Premium Fast-charging Portable Charger',
   'Chromcraft Rectangular Conference Tables',
   'Hon Every-Day Series Multi-Task Chairs',
   'Global Low Back Tilter Chair',
   'Eldon Image Series Desk Accessories',
   ' Ebony',
   'Acco Economy Flexible Poly Round Ring Binder'],
  'quantity': Decimal('100')},
 {'name': 'Greg Matthias',
  'product_name': ['Office Star - Task Chair with Contemporary Loop Arms',
   'Economy #2 Pencils',
   'Avery 52',
   'Avery 476',
   'Novimex High-Tech Fabric Mesh Task Chair',
   'SanDisk Cruzer 4 GB USB Flash Drive'],
  'quantity': Decimal('100')},
 {'name': 'Chad McG

In [3]:
from mysql.connector import connect

db = connect(host = db_host, 
             user = db_user, 
             password = params['db']['MYSQL_PASSWORD'],
             database = db_name)

#cek koneksi database
try:
    print(db)
    print('Database berhasil terkoneksi')
except:
    print('Database tidak terkoneksi')

Database berhasil terkoneksi


In [4]:
def query_barang_tanggal(db,nama_barang,tanggal_awal,tanggal_akhir):
    query = f'''
            select DATE_FORMAT(STR_TO_DATE(a.date, '%d-%m-%Y'), '%d-%m-%Y') AS tanggal, 
            a.name, a.product_name, c.category, c.sub_category, a.quantity, b.city, b.state
            from purchase a
            inner join users b on a.name = b.name
            inner join products c on a.product_name = c.product_name
            where a.product_name Like '%{nama_barang}%'
            and STR_TO_DATE(a.date, '%d-%m-%Y') 
            BETWEEN STR_TO_DATE('{tanggal_awal}', '%d-%m-%Y') AND STR_TO_DATE('{tanggal_akhir}', '%d-%m-%Y')
            ORDER BY a.date ASC, a.quantity DESC, a.name ASC;
            '''
    cursor_db = db.cursor()
    cursor_db.execute(query)
    result = cursor_db.fetchall()
    hasil = []
    for i in result:
        kucing = {}
        kucing['Tanggal'] = i[0]
        kucing['Nama Pembeli'] = i[1]
        kucing['Nama Produk'] = i[2]
        kucing['Kategori'] = i[3]
        kucing['Sub Kategori'] = i[4]
        kucing['Kuantitas'] = i[5]
        kucing['kota'] = i[6]
        kucing['Negara'] = i[7]
        hasil.append(kucing)
    cursor_db.close()
    db.close()
    return hasil
    
# query_barang_tanggal(db,nama_barang,tanggal_awal,tanggal_akhir)
query_barang_tanggal(db,'usb','01-01-2020','31-12-2020')

[{'Tanggal': '13-05-2020',
  'Nama Pembeli': 'Jennifer Jackson',
  'Nama Produk': 'Imation\xa08gb Micro Traveldrive Usb 2.0\xa0Flash Drive',
  'Kategori': 'Technology',
  'Sub Kategori': 'Accessories',
  'Kuantitas': 21,
  'kota': 'College Station',
  'Negara': 'Texas'},
 {'Tanggal': '20-12-2020',
  'Nama Pembeli': 'Bobby Odegard',
  'Nama Produk': 'Imation\xa08gb Micro Traveldrive Usb 2.0\xa0Flash Drive',
  'Kategori': 'Technology',
  'Sub Kategori': 'Accessories',
  'Kuantitas': 24,
  'kota': 'Stockton',
  'Negara': 'California'},
 {'Tanggal': '20-12-2020',
  'Nama Pembeli': 'Anthony Witt',
  'Nama Produk': 'Belkin F8E887 USB Wired Ergonomic Keyboard',
  'Kategori': 'Technology',
  'Sub Kategori': 'Accessories',
  'Kuantitas': 5,
  'kota': 'Provo',
  'Negara': 'Utah'},
 {'Tanggal': '24-01-2020',
  'Nama Pembeli': 'Sanjit Chand',
  'Nama Produk': 'Dell Slim USB Multimedia Keyboard',
  'Kategori': 'Technology',
  'Sub Kategori': 'Accessories',
  'Kuantitas': 10,
  'kota': 'Romeoville',

In [2]:
import jwt

In [5]:
import pandas as pd
import random
random.seed(7)
from datetime import datetime, timedelta
import urllib.parse
from dotenv import dotenv_values
from mysql.connector import connect

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime, distinct, and_
from sqlalchemy.ext.declarative import declarative_base

from fastapi import FastAPI, File, UploadFile, HTTPException, Depends
import uvicorn
import nest_asyncio
nest_asyncio.apply()

import jwt
from fastapi.security import HTTPBearer, HTTPAuthorizationCredentials
from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.openapi.utils import get_openapi

params = {'db':dotenv_values('TugasAkhir.env')}

db_user = params['db']['MYSQL_USERNAME']
db_password = urllib.parse.quote_plus(params['db']['MYSQL_PASSWORD'])
db_host = params['db']['MYSQL_HOST']
db_port = params['db']['MYSQL_PORT']
db_name = params['db']['MYSQL_DB']

# Buat koneksi ke database MySQL
SQLALCHEMY_DATABASE_URL = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(SQLALCHEMY_DATABASE_URL, echo=True, pool_pre_ping=True)

# Membuat tanggal acak
def random_tanggal():
    # Set the start and end dates
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2022, 12, 17)
    # Generate a random date between the start and end dates
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    # Convert the random date to a string in the DD--MM-YYYY format
    formatted_date = random_date.strftime("%d-%m-%Y")
    return formatted_date

# Buat base class untuk ORM
Base = declarative_base()

try:
    # Buat tabel products
    class Products(Base):
        __tablename__ = 'products'
        product_id = Column(Integer, nullable=False)
        product_name = Column(String(50), primary_key=True)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)

    # Buat tabel users
    class Users(Base):
        __tablename__ = 'users'
        customer_id = Column(Integer, nullable=False)
        name = Column(String(50), primary_key=True)
        city = Column(String(50), nullable=False)
        state = Column(String(50), nullable=False)
        postal = Column(Integer, nullable=False)

    # Buat tabel purchase
    class Purchase(Base):
        __tablename__ = 'purchase'
        name = Column(String(50), nullable=False, primary_key=True)
        date = Column(DateTime, nullable=False)
        product_name = Column(String(50), nullable=False, primary_key=True)
        quantity = Column(Integer, nullable=False)

    Base.metadata.create_all(engine)
except Exception as e:
    print(e)        

app = FastAPI()

def tabel_sql_jadi(users_csv,products_csv):
    df_users = pd.read_csv(users_csv.file)
    df_products = pd.read_csv(products_csv.file, sep=';', encoding='latin-1')
    df_products.columns = ['product_id', 'product_name', 'category', 'sub_category']

    n = 1000 #banyaknya kolom tabel purchase

    df_purchase = pd.DataFrame({'date':[random_tanggal() for i in range(n)],
                         'name':[random.choices(df_users['name'])[0] for i in range(n)], 
                         'product_name':[random.choices(df_products['product_name'])[0] for i in range(n)],
                         'quantity':[random.randint(1,25) for i in range(n)]
                        })
    
    sudah_menambahkan_data_pada_sql = True # Jawab True atau False
    
    if sudah_menambahkan_data_pada_sql:
        pass
    else:
        # Insert data ke tabel products
        df_products.to_sql('products', engine, if_exists='replace', index=False)

        # Insert data ke tabel users
        df_users.to_sql('users', engine, if_exists='replace', index=False)

        # Insert data ke tabel purchase
        df_purchase.to_sql('purchase', engine, if_exists='replace', index=False)
    
# Buat Session
Session = sessionmaker(bind=engine)
session = Session()
conn = engine.connect()

# process JWT
security = HTTPBearer()

def create_jwt(data):
    # Set kunci yang akan digunakan untuk men-encode token
    secret = "my-secret"
    # Set header dan payload token
    header = {"alg": "HS256", "typ": "JWT"}
    payload = {"sub": data}
    # Encode token dan kembalikan hasilnya
    return jwt.encode(header, payload, secret)

def validate_jwt(token):
    # Set kunci yang akan digunakan untuk men-decode token
    secret = "kucing"
    try:
        # Coba decode token dengan kunci yang telah ditentukan
        data = jwt.decode(token, secret)
        # Kembalikan informasi pengguna yang terkandung dalam token
        return data["sub"]
    except:
        # Jika decode gagal (misalnya karena token tidak valid), kembalikan nilai None
        return None

@app.get("/login")
async def login(username: str, password: str):
    # Coba authentikasi pengguna dengan username dan password yang diterima
    if authenticate(username, password):
        # Jika autentikasi berhasil, buat token JWT baru dan kembalikan ke pengguna
        token = create_jwt({"username": username})
        return {"access_token": token}
    else:
        # Jika autentikasi gagal, kembalikan pesan kesalahan
        return {"error": "Invalid username or password"}

@app.get("/protected")
@security.requires_auth()
async def protected(credentials: HTTPAuthorizationCredentials = Depends(security)):
    # Validasi token
    user = validate_jwt(credentials.credentials)
    if user:
        # Jika token valid, kembalikan pesan untuk menandakan bahwa pengguna berhasil login
        return {"message": f"Hello, {user['username']}! You have access to this protected feature."}
    else:
        # Jika token tidak valid, kembalikan pesan kesalahan
        raise HTTPException(status_code=401, detail="Invalid authorization token")

        
        
        
# Menyapa pada localhost awal
@app.get("/")
async def tugas_akhir():
    return '''Halo, saya uzu.
                Ini merupakan tugas akhir dari 
                Sanbercode Python Backend Development'''

# Menampilkan isi dari tabel (users, products, purchase)
@app.post("/tabel/{nama_tabel}")
@security.requires_auth()
async def tampilkan_isi_tabel(users_csv: UploadFile = File(...), 
                          products_csv: UploadFile = File(...),
                          nama_tabel = str
                         ):
    tabel_sql_jadi(users_csv,products_csv)
    
    return conn.execute(f'select * from {nama_tabel.lower()}').all()

    session.commit()
    session.close()

# Menampilkan semua barang dengan urutan ascending (None, True, False)
@app.post("/barang")
async def tampilkan_barang(users_csv: UploadFile = File(...), 
                          products_csv: UploadFile = File(...),
                          sort_desc = None
                         ):
    tabel_sql_jadi(users_csv,products_csv)
    
    result = session.query(Products.product_name).all()
    session.commit()
    
    produk = [i[0] for i in result]
    if sort_desc==None:
        pass
    elif sort_desc.lower() == 'true':
        produk.sort(reverse=False)
    elif sort_desc.lower() == 'false':
        produk.sort(reverse=True)
    return produk

    session.commit()
    session.close()

# Menampilkan semua barang yang bernama <nama_barang> dengan urutan ascending (None, True, False)
@app.post("/search_barang/{nama_barang}")
async def tampilkan_barang(users_csv: UploadFile = File(...), 
                          products_csv: UploadFile = File(...),
                          nama_barang = '',
                          sort_desc = None
                         ):
    tabel_sql_jadi(users_csv,products_csv)
    
    result = session.query(Products.product_name).all()
    session.commit()

    produk = [i[0] for i in result]
    cari_barang = []
    for i in produk:
        if nama_barang.lower() in i.lower():
             cari_barang.append(i)

    if sort_desc==None:
        pass
    elif sort_desc.lower() == 'true':
        cari_barang.sort(reverse=False)
    elif sort_desc.lower() == 'false':
        cari_barang.sort(reverse=True)
    return cari_barang

    session.commit()
    session.close()

# Menampilkan semua orang <nama_pembeli> berserta kota dan negaranya dengan urutan ascending (None, True, False)
@app.post("/pembeli")
@security.requires_auth()
async def tampilkan_nama_tinggal(users_csv: UploadFile = File(...), 
                                  products_csv: UploadFile = File(...),
                                  sort_desc = None
                                 ):
    tabel_sql_jadi(users_csv,products_csv)
    
    if sort_desc==None:
        result = (session.query(sa.distinct(Users.name), 
                            Users.city, 
                            Users.state))
    if sort_desc=='true':
        result = (session.query(sa.distinct(Users.name), 
                            Users.city, 
                            Users.state)
        .order_by(Users.name.desc()))
    if sort_desc=='false':
        result = (session.query(sa.distinct(Users.name), 
                            Users.city, 
                            Users.state)
        .order_by(Users.name.asc()))
    
    hasil = []
    for x, i in enumerate(result):
        hasil.append(f'{x+1}. {i[0]} {i[1]} {i[2]}')
    return hasil

    session.commit()
    session.close()

    
# Menampilkan <n> orang dengan total pembelian terbanyak berserta nama barangnya
@app.post("/pembeli_terbanyak")
@security.requires_auth()
async def tampilkan_nama_tinggal(users_csv: UploadFile = File(...), 
                                  products_csv: UploadFile = File(...),
                                  banyaknya_orang_yang_ditampilkan = 1000
                                 ):
    tabel_sql_jadi(users_csv,products_csv)
    
    n = int(banyaknya_orang_yang_ditampilkan)
    
    result = (
    session.query(Purchase.name, 
                  sa.func.group_concat(Purchase.product_name, separator=", ").label("product_name"), 
                  sa.func.sum(Purchase.quantity).label("quantity"))
        .group_by(Purchase.name)
        .order_by(sa.func.sum(Purchase.quantity).desc())
        .limit(n)
        )

    hasil = []
    for i in result:
        kucing = {}
        kucing['name'] = i[0]
        kucing['product_name'] = i[1].split(',')
        kucing['quantity'] = i[2]
        hasil.append(kucing)
    
    return hasil
    
    session.commit()
    session.close()    
    
    
# Menampilkan semua data pembeli dan barang dengan query parameternya adalah nama barang, tanggal awal pembelian dan tanggal akhir pembelian
@app.post("/tampilkan_semua")
@security.requires_auth()
async def tampilkan_semua(users_csv: UploadFile = File(...), 
                                  products_csv: UploadFile = File(...),
                                  nama_barang = None,
                                  tanggal_awal = '01-01-2017',
                                  tanggal_akhir = '17-12-2022'
                                 ):
    tabel_sql_jadi(users_csv,products_csv)
    
    db = connect(host = db_host, 
         user = db_user, 
         password = params['db']['MYSQL_PASSWORD'],
         database = db_name)
    
    def query_barang_tanggal(db,nama_barang,tanggal_awal,tanggal_akhir):
        query = f'''
                select DATE_FORMAT(STR_TO_DATE(a.date, '%d-%m-%Y'), '%d-%m-%Y') AS tanggal, 
                a.name, a.product_name, c.category, c.sub_category, a.quantity, b.city, b.state
                from purchase a
                inner join users b on a.name = b.name
                inner join products c on a.product_name = c.product_name
                where a.product_name Like '%{nama_barang}%'
                and STR_TO_DATE(a.date, '%d-%m-%Y') 
                BETWEEN STR_TO_DATE('{tanggal_awal}', '%d-%m-%Y') AND STR_TO_DATE('{tanggal_akhir}', '%d-%m-%Y')
                ORDER BY a.date ASC, a.quantity DESC, a.name ASC;
                '''
        cursor_db = db.cursor()
        cursor_db.execute(query)
        result = cursor_db.fetchall()
        hasil = []
        for i in result:
            kucing = {}
            kucing['Tanggal'] = i[0]
            kucing['Nama Pembeli'] = i[1]
            kucing['Nama Produk'] = i[2]
            kucing['Kategori'] = i[3]
            kucing['Sub Kategori'] = i[4]
            kucing['Kuantitas'] = i[5]
            kucing['kota'] = i[6]
            kucing['Negara'] = i[7]
            hasil.append(kucing)
        cursor_db.close()
        db.close()
        return hasil

    return query_barang_tanggal(db,nama_barang,tanggal_awal,tanggal_akhir)
    
if __name__ == "__main__":
    uvicorn.run(app)

2022-12-23 19:15:09,559 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-12-23 19:15:09,559 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 19:15:09,562 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-12-23 19:15:09,563 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 19:15:09,565 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-12-23 19:15:09,566 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-12-23 19:15:09,568 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-23 19:15:09,569 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 19:15:09,570 INFO sqlalchemy.engine.Engine [generated in 0.00146s] {'table_schema': 'penjualan_uzu', 'table_name': 'products'}
2022-12-23 19:15:09,573 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-12-23 19:15:0

AttributeError: 'HTTPBearer' object has no attribute 'requires_auth'

In [ ]:
# process JWT
security = HTTPBearer()

def create_jwt(data):
    # Set kunci yang akan digunakan untuk men-encode token
    secret = "my-secret"
    # Set header dan payload token
    header = {"alg": "HS256", "typ": "JWT"}
    payload = {"sub": data}
    # Encode token dan kembalikan hasilnya
    return jwt.encode(header, payload, secret)

def validate_jwt(token):
    # Set kunci yang akan digunakan untuk men-decode token
    secret = "kucing"
    try:
        # Coba decode token dengan kunci yang telah ditentukan
        data = jwt.decode(token, secret)
        # Kembalikan informasi pengguna yang terkandung dalam token
        return data["sub"]
    except:
        # Jika decode gagal (misalnya karena token tidak valid), kembalikan nilai None
        return None

@app.get("/login")
async def login(username: str, password: str):
    # Coba authentikasi pengguna dengan username dan password yang diterima
    if authenticate(username, password):
        # Jika autentikasi berhasil, buat token JWT baru dan kembalikan ke pengguna
        token = create_jwt({"username": username})
        return {"access_token": token}
    else:
        # Jika autentikasi gagal, kembalikan pesan kesalahan
        return {"error": "Invalid username or password"}

@app.get("/protected")
@security.requires_auth
async def protected(credentials: HTTPAuthorizationCredentials = Depends(security)):
    # Validasi token
    user = validate_jwt(credentials.credentials)
    if user:
        # Jika token valid, kembalikan pesan untuk menandakan bahwa pengguna berhasil login
        return {"message": f"Hello, {user['username']}! You have access to this protected feature."}
    else:
        # Jika token tidak valid, kembalikan pesan kesalahan
        raise HTTPException(status_code=401, detail="Invalid authorization token")
        
AttributeError: 'HTTPBearer' object has no attribute 'requires_auth'